In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-07"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13670,2024-11-07,Brasil Nbb,19:00,Minas,Fortaleza B.C.,1.11,5.17,150.5,1.80,1.86,-13.5,1.95,1.72,noKberoS,0.900901,0.193424,0.555556,0.537634,0.094324,0.2,0.4,NaN,NaN,108.016,15.020395,0.139057,155.984,57.647152,0.369571,108.264,151.078,94.64,194.40,0.0,0.0,0.0,0.0,0.914285,0.023184,0.088629,-0.07,-0.014,-7.857143,0.783221,0.8,0.016779,-1.64,-0.328,-12.713415,0.395112,0.3,-0.095112
13671,2024-11-07,Brasil Nbb,20:00,R10 Score Vasco da Gama,Caxias do Sul,1.12,4.97,144.5,1.83,1.83,-12.5,2.02,1.67,OQDxl0w2,0.892857,0.201207,0.546448,0.546448,0.094064,0.6,0.2,NaN,NaN,122.080,41.603570,0.340789,298.274,184.161611,0.617424,126.780,328.094,83.70,577.50,0.0,1.0,0.0,0.0,0.894043,0.000000,0.134139,-0.79,-0.158,-0.759494,0.620248,0.7,0.079752,-0.49,-0.098,-40.510204,0.301173,0.2,-0.101173
13672,2024-11-07,Brasil Nbb,20:30,São Paulo,Flamengo,2.86,1.35,155.5,1.83,1.83,5.5,1.97,1.71,jDFUkM8k,0.349650,0.740741,0.546448,0.546448,0.090391,0.4,0.4,NaN,NaN,135.612,53.024763,0.391003,86.928,15.800940,0.181770,138.312,98.830,91.08,109.62,0.0,0.0,0.0,0.0,0.507236,0.000000,0.099917,0.89,0.178,10.449438,0.600721,0.5,-0.100721,1.13,0.226,1.548673,0.791877,0.9,0.108123
13673,2024-11-07,Espanha Acb,16:30,Breogan,Tenerife,2.67,1.39,155.5,1.86,1.94,4.5,1.96,1.72,AJsPDeH0,0.374532,0.719424,0.537634,0.515464,0.093956,0.4,0.8,NaN,NaN,191.386,59.594251,0.311383,154.926,44.284929,0.285846,208.218,149.754,117.81,127.28,0.0,0.0,0.0,0.0,0.445860,0.029773,0.092231,-3.45,-0.690,-2.420290,0.531645,0.4,-0.131645,1.41,0.282,1.382979,0.498926,0.7,0.201074
13674,2024-11-07,Eua Nba,00:00,Los Angeles Clippers,Philadelphia 76ers,1.85,2.02,213.5,1.81,2.04,-2.5,2.04,1.81,lInWf2fK,0.540541,0.495050,0.552486,0.490196,0.035590,0.6,0.6,NaN,NaN,266.638,143.220046,0.537133,233.956,97.898112,0.418447,227.288,263.002,177.41,390.92,0.0,0.0,0.0,0.0,0.062123,0.084485,0.084485,-3.43,-0.686,-1.239067,0.571319,0.6,0.028681,-1.13,-0.226,-4.513274,0.454975,0.5,0.045025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13750,2024-11-07,Europa Eurobasket Feminino,16:45,Bélgica F,Lituânia F,1.02,11.60,140.5,1.87,1.86,-23.5,1.92,1.79,fXlBDQMF,0.980392,0.086207,0.534759,0.537634,0.066599,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,63.24,300.00,0.0,0.0,1.0,0.0,1.185609,0.003791,0.049555,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
13751,2024-11-07,Europa Eurobasket Feminino,17:00,Espanha F,Países Baixos F,1.02,12.50,128.5,1.87,1.86,-27.5,1.99,1.73,dQWOiq0d,0.980392,0.080000,0.534759,0.537634,0.060392,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,75.75,143.07,0.0,0.0,0.0,0.0,1.200826,0.003791,0.098843,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
13752,2024-11-07,Europa Eurobasket Feminino,17:10,França F,Israel F,1.02,12.50,139.5,1.88,1.78,-28.5,2.05,1.68,nVTk6rFk,0.980392,0.080000,0.531915,0.561798,0.060392,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,89.46,0.00,0.0,0.0,0.0,0.0,1.200826,0.038640,0.140284,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
13753,2024-11-07,México Lnbp,23:00,Diablos Rojos,Soles,1.58,2.18,178.5,1.88,1.78,-4.5,1.98,1.70,fkbUMmsI,0.632911,0.458716,0.531915,0.561798,0.091627,0.0,0.0,NaN,NaN,133.000,27.650272,0.207897,161.346,53.430663,0.3311

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Los Angeles Clippers,Philadelphia 76ers,213.5,1.81,0.8581,Over
1,00:00,Eua Nba,Sacramento Kings,Toronto Raptors,236.5,1.84,0.8833,Over
2,16:30,Europa Euroliga,Milano,Real Madrid,162.5,1.91,0.7190,Over
3,16:45,Europa Euroliga,Paris,Partizan,168.5,1.91,0.7358,Over
4,21:00,Eua Ncaa,Wake Forest,N. Carolina A&T,148.5,1.91,0.9122,Over
